## Q1. Getting the embeddings model

First, we will get the embeddings model `multi-qa-distilbert-cos-v1` from
[the Sentence Transformer library](https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview)

```bash
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(model_name)
```

Create the embedding for this user question:

```python
user_question = "I just discovered the course. Can I still join it?"
```

What's the first value of the resulting vector?

* -0.24
* -0.04
* 0.07
* 0.27


## Prepare the documents

Now we will create the embeddings for the documents.

Load the documents with ids that we prepared in the module:

```python
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()
```

We will use only a subset of the questions - the questions
for `"machine-learning-zoomcamp"`. After filtering, you should
have only 375 documents

In [1]:
from sentence_transformers import SentenceTransformer

# Load the pre-trained model
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

# User question
user_question = "I just discovered the course. Can I still join it?"

# Create embedding for the user question
embedding = embedding_model.encode(user_question)
# Get the first value of the resulting vector
first_value = embedding[0]
print(first_value)

/home/alexei/miniconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


0.07822263


**Answer:** 0.07

In [2]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [3]:
ml_zoomcamp_docs = [doc for doc in documents if doc["course"] == "machine-learning-zoomcamp"]

In [4]:
ml_zoomcamp_docs[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

## Q2. Creating the embeddings

Now for each document, we will create an embedding for both question and answer fields.

We want to put all of them into a single matrix `X`:

- Create a list `embeddings` 
- Iterate over each document 
- `qa_text = f'{question} {text}'`
- compute the embedding for `qa_text`, append to `embeddings`
- At the end, let `X = np.array(embeddings)` (`import numpy as np`) 

What's the shape of X? (`X.shape`). Include the parantheses.

In [8]:
embeddings = []
import numpy as np

In [9]:
# Iterate over each document
for doc in ml_zoomcamp_docs:
    qa_text = f'{doc["question"]} {doc["text"]}'
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

In [32]:
# Convert the list of embeddings to a NumPy array
X = np.array(embeddings)

# Output the shape of X
X_shape = X.shape
X_shape

(375, 768)

**Answer:** (375, 768)

## Q3. Search

We have the embeddings and the query vector. Now let's compute the 
cosine similarity between the vector from Q1 (let's call it `v`) and the matrix from Q2. 

The vectors returned from the embedding model are already
normalized (you can check it by computing a dot product of a vector
with itself - it should return something very close to 1.0). This means that in order
to compute the coside similarity, it's sufficient to 
multiply the matrix `X` by the vector `v`:


```python
scores = X.dot(v)
```

What's the highest score in the results?

- 65.0 
- 6.5
- 0.65
- 0.065


## Vector search

We can now compute the similarity between a query vector and all the embeddings.

Let's use this to implement our own vector search

```python
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)
```

If you don't understand how the `search` function work:

* Ask ChatGTP or any other LLM of your choice to explain the code
* Check our pre-course workshop about implementing a search engine [here](https://github.com/alexeygrigorev/build-your-own-search-engine)

(Note: you can replace `argsort` with `argpartition` to make it a lot faster)

In [12]:
# User question
user_question = "I just discovered the course. Can I still join it?"

# Create embedding for the user question
v = embedding_model.encode(user_question)

In [14]:
scores = X.dot(v)
np.max(scores)

0.65065724

**Answer:** 0.65

## Q4. Hit-rate for our search engine

Let's evaluate the performance of our own search engine. We will
use the hitrate metric for evaluation.

First, load the ground truth dataset:

```python
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')
```

Now use the code from the module to calculate the hitrate of
`VectorSearchEngine` with `num_results=5`.

What did you get?

* 0.93
* 0.73
* 0.53
* 0.33

In [18]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [49]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [22]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [33]:
# Initialize the search engine
search_engine = VectorSearchEngine(documents=ml_zoomcamp_docs, embeddings=X)

In [34]:
# Function to calculate hitrate
def calculate_hitrate(search_engine, ground_truth, num_results=5):
    hits = 0
    for item in ground_truth:
        query = item['question']
        true_document_id = item['document']
        query_embedding = embedding_model.encode(query)
        results = search_engine.search(query_embedding, num_results=num_results)
        if any(result['id'] == true_document_id for result in results):
            hits += 1
    return hits / len(ground_truth)

In [35]:
calculate_hitrate(search_engine, ground_truth, num_results=5)

0.9398907103825137

**Answer:** 0.93

## Q5. Indexing with Elasticsearch

Now let's index these documents with elasticsearch

* Create the index with the same settings as in the module (but change the dimensions)
* Index the embeddings (note: you've already computed them)

After indexing, let's perform the search of the same query from Q1.

What's the ID of the document with the highest score?

In [82]:
from elasticsearch import Elasticsearch

In [83]:
es = Elasticsearch('http://localhost:9200')

In [88]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 1
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "text"},
            "embedding": {"type": "dense_vector", "dims": 768}
        }
    }
}

In [85]:
# Create the index
es.indices.create(index="ml_zoomcamp_index", body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ml_zoomcamp_index'})

In [44]:
# Assuming you have a list of documents and corresponding embeddings
for doc, emb in zip(ml_zoomcamp_docs, embeddings):
    es.index(index="ml_zoomcamp_index", body={
        "text": doc["text"],
        "section": doc["section"],
        "question": doc["question"],
        "course": doc["course"],
        "embedding": emb.tolist()  # Convert NumPy array to list
    })

In [53]:
query = "I just discovered the course. Can I still join it?"
query_embedding = embedding_model.encode(query).tolist()

# Define the search query
search_query = {
    "query": {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                "params": {"query_vector": query_embedding}
            }
        }
    }
}

# Perform the search
results = es.search(index="ml_zoomcamp_index", body=search_query)

# Print the ID of the document with the highest score
top_hit = results['hits']['hits'][0]
print(f"Document ID: {top_hit['_id']}, Score: {top_hit['_score']}")

Document ID: Ml4GwJAB6701txdGLBcz, Score: 1.6506574


**Answer:** Document ID: Ml4GwJAB6701txdGLBcz

## Q6. Hit-rate for Elasticsearch

The search engine we used in Q4 computed the similarity between
the query and ALL the vectors in our database. Usually this is 
not practical, as we may have a lot of data.

Elasticsearch uses approximate techniques to make it faster. 

Let's evaluate how worse the results are when we switch from
exact search (as in Q4) to approximate search with Elastic.

What's hitrate for our dataset for Elastic?

* 0.93
* 0.73
* 0.53
* 0.33

In [74]:
def calculate_text_based_hitrate(ground_truth, index_name, num_results=5):
    hits = 0
    total = 0

    for item in ground_truth:
        query_text = item['question']
        true_document_id = item['document']

        search_query = {
            "size": num_results,
            "query": {
                "bool": {
                    "must": {
                        "multi_match": {
                            "query": query_text,
                            "fields": ["question^3", "text", "section"],
                            "type": "best_fields"
                        }
                    },
                    "filter": {
                        "term": {
                            "course": "machine-learning-zoomcamp"
                        }
                    }
                }
            }
        }

        results = es.search(index=index_name, body=search_query)
        result_ids = [hit['_id'] for hit in results['hits']['hits']]

        if true_document_id in result_ids:
            hits += 1
        total += 1

    return hits / total

In [75]:
hitrate_text_based = calculate_text_based_hitrate(ground_truth, "ml_zoomcamp_index")
print(f"Text-based hitrate for Elasticsearch: {hitrate_text_based:.2f}")

Text-based hitrate for Elasticsearch: 0.00
